# DETCTING AND DONWLOADING LATEST/PREVIOUS EN CHAPTER
Preferrably on release

In [1]:
# Author				: G.M. Yongco #BeSomeoneWhoCanStandByShinomiya
# Date					: ur my date uwu
# Description			: Code that will impress u ;)
# Actual Description	: captures the manga list, filters, sorts, then downloads manga
# Reference				: https://api.mangadex.org/docs/

## Imports we will be using and utility functions that are used in other functions

In [2]:
import requests
import json
import os

def write_json(file_name:str = "template_json.text", content = "{'template':'true'}") -> bool:
    try:
        with open(file_name, "w", encoding="utf-8") as f:
            json.dump(content, f, indent=4, ensure_ascii=False)
        return True
    except Exception as e:
        print(e)
        return False


### Functions to get the chapter list of a certain manga

In [3]:
def get_general_info(manga_title:str = "The Guy She Was Interested in Wasn't a Guy at All") -> dict:
    base_url = "https://api.mangadex.org"

    response:json = requests.get(
        url = f"{base_url}/manga",
        params={"title": manga_title}
    ).json()

    with open("get_general_info.txt", "w", encoding="utf-8") as f:
        json.dump(response, f, indent=4, ensure_ascii=False)

    return response

def get_chapters(manga_title:str = "The Guy She Was Interested in Wasn't a Guy at All") -> dict:
    base_url = "https://api.mangadex.org"
    manga_id = get_general_info(manga_title=manga_title)['data'][0]['id']
    languages = ["en"]
    
    response:json = requests.get(
        url = f"{base_url}/manga/{manga_id}/feed",
        params={"translatedLanguage[]": languages, 
                "limit": 200}
    ).json()

    if response['result'] == "ok":
        print("function - success - get_chapters()")
    else:
        print("function - fail - get_chapters()")

    return response

## Now we put the output in a text file for humans to check

In [4]:
# chapters:list = get_chapters("Amasaya")
chapters:list = get_chapters()
print(write_json(file_name = "get_chapters_response.txt", content = chapters))

function - success - get_chapters()
True


### Formatting

In [5]:
chapters_formatted = chapters["data"]
chapters_formatted = list(filter(lambda x: x["attributes"]["translatedLanguage"] == "en", chapters_formatted))
chapters_formatted = list(sorted(chapters_formatted, key = lambda x: float(x["attributes"]["chapter"])))
print(write_json(file_name = "get_chapters_response_formatted.txt", content = chapters_formatted))


True


### Check for missing chapters

In [6]:
chapter_num:set = []
for chapter in chapters_formatted:
    chapter_num.append(float(chapter["attributes"]["chapter"]))

full_range = set(range(int(min(chapter_num)), int(max(chapter_num)) + 1))
missing = sorted(full_range - set(chapter_num))

print(missing)
print(len(missing))

[]
0


## Downloading

In [7]:
def get_chapter_pannels(chapter_id:str = "template-id")->bool:
    base_url = "https://api.mangadex.org"

    response:json = requests.get(
        url = f"{base_url}/at-home/server/{chapter_id}"
    ).json()

    if response['result'] == "ok":
        print("function - success - get_chapter_pannels()")
    else:
        print("function - fail - get_chapter_pannels()")

    return response

random_chapter_for_testing = 42
chapter_response:list = get_chapter_pannels(chapters_formatted[random_chapter_for_testing]["id"])
print(write_json(file_name = "get_chapter_pannels.txt", content = chapter_response))

function - success - get_chapter_pannels()
True


### Now we extract the download ready image links from the response 
using this format:<br>
[baseUrl]/[quality]/[chapterHash]/[filename]

In [8]:
image_links:list = []
baseUrl:str = chapter_response['baseUrl']
quality:str = "data"
chapterHash:str = chapter_response['chapter']['hash']
for file_name in chapter_response["chapter"]["data"]:
    image_links.append(f"{baseUrl}/{quality}/{chapterHash}/{file_name}")

print(write_json(file_name = "pannel_links.txt", content = image_links))

True


### Now we make a download function and test it with one of the links

In [13]:
def download_img(
		url:str = "https://cmdxd98sb0x3yprd.mangadex.network/data/c4db4703e60ba5e812b9dca733284c81/4-20dcc4e997f9762f47292c68a26d6f3165ec9ff66dd41899a322d5a5afd876c7.png", 
		file_name:str = "a.png"
	) -> None:
	path:str = "DOWNLOADS/"
	if(os.path.exists(path) == False):
		os.makedirs(path)

	path += file_name

	try:
		response:requests.Response = requests.get(
            url, stream=True
        )
		response.raise_for_status()  # Raise an exception for error responses

		with open(path, 'wb') as f:
			for chunk in response.iter_content(chunk_size=8192):
				if chunk:  # Filter out keep-alive new chunks
					f.write(chunk)
	except Exception as e:
		print(e)
	else:
	    print("Image downloaded successfully!")

In [14]:
download_img(url = image_links[3], file_name="test.jpg")

Image downloaded successfully!
